# **'Precio Media Anual'** Propiedades

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "pachuca" in nombre
    and ('financiamientos' not in nombre and 'poblacion' not in nombre and 'salarios' not in nombre and 'publicacion' not in nombre and 'q' not in nombre)
    or ('df_alfa_q_sep_2023_pachuca'in nombre)
    or ('df_alfa_q_nov_2022_pachuca'in nombre or 'df_alfa_q_oct_2022_pachuca'in nombre )
]  

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_pachuca',
 'df_alfa_agosto_2024_pachuca',
 'df_alfa_febrero_2024_pachuca',
 'df_alfa_julio_2024_pachuca',
 'df_alfa_junio_2024_pachuca',
 'df_alfa_marzo_2024_pachuca',
 'df_alfa_mayo_2024_pachuca',
 'df_alfa_q_nov_2022_pachuca',
 'df_alfa_q_oct_2022_pachuca',
 'df_alfa_q_sep_2023_pachuca']

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
        # Renombrar la columna
    df.rename(columns={'Recamaras':'recamaras','Estacionamiento':'estacionamiento','m_total': 'metros_total','m_construido':'metros_construido','tiempo_publicacion':'tiempo_de_publicacion','Estacionamientos':'estacionamiento','Recamaras': 'recamaras', 
       'baño':'Banos','medio_baño':'Medio_banos', 'Seguridad':'seguridad','seguridad_privada':'seguridad','baño_total':'Banos_Total', 'cp':'CP',                  'Colonia':'colonia','Estado':'estado','Tipo':'tipo','Categoria':'categoria','Precio':'precio','m2_contruido': 'm2_construido','segmento':'categoria','Categoria':'categoria','Category':'categoria'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")

Columnas renombradas en los DataFrames filtrados.


In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
dataframes_name = []

for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['categoria', 'precio']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)
     # Componer el nombre del DataFrame en la lista dataframes_name
    dataframes_name.append(nombre_df)

In [21]:
import re

# Función para extraer mes y año del nombre del DataFrame
def extraer_mes_y_ano(nombre):
    # Expresión regular para capturar tanto meses completos como abreviados
    match = re.search(r'(ene|feb|mar|abr|may|jun|jul|ago|sep|oct|nov|dic|enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)_(\d{4})', nombre)
    if match:
        mes = match.group(1).lower()  # Mes en texto
        ano = int(match.group(2))  # Año en formato numérico
        return mes, ano
    return None, None

# Diccionario de traducción de meses de texto a números
meses_dict = {
    'ene': 1, 'feb': 2, 'mar': 3, 'abr': 4, 'may': 5, 'jun': 6,
    'jul': 7, 'ago': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dic': 12,
    'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4, 'mayo': 5, 'junio': 6,
    'julio': 7, 'agosto': 8, 'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
}

df_resultados = pd.DataFrame(columns=['categoria', 'precio', 'mes', 'año'])
for nombre_df in nombres_df_filtrados:
    df = globals()[nombre_df]
    
    # Extraer mes y año del nombre del DataFrame
    mes_str, ano = extraer_mes_y_ano(nombre_df)
    
    # Verificar que se haya extraído el mes y el año correctamente
    if mes_str is not None and ano is not None:
        mes = meses_dict.get(mes_str)  # Convertir el nombre del mes en su número correspondiente
        
        # Añadir las columnas de mes y año al DataFrame actual
        df['mes'] = mes
        df['año'] = ano
        df_resultados = pd.concat([df_resultados, df], ignore_index=True)
df_resultados[['categoria','precio','mes','año']]


,categoria,precio,mes,año
0,S3,8500000,4,2024
1,S1,4500000,4,2024
2,S3,8600000,4,2024
3,S3,8600000,4,2024
4,E2,615000,4,2024
...,...,...,...,...
16857,S,999999.99,9,2023
16858,S,999999.99,9,2023
16859,B3,999999.99,9,2023
16860,S,999999.99,9,2023


# Regresión mensual (RF)

In [5]:
df_resultados = df_resultados.groupby(['mes','año'])['precio'].median().reset_index()
df_resultados.rename(columns={'precio': 'montos'}, inplace=True)

df_resultados

,mes,año,montos
0,2,2024,2500000.0
1,3,2024,2500000.0
2,4,2024,2300000.0
3,5,2024,2250000.0
4,6,2024,2300000.0
5,7,2024,2400000.0
6,8,2024,2500000.0
7,9,2023,999999.99
8,10,2022,999999.99
9,11,2022,999999.99


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np

X = df_resultados[['año', 'mes']]
y = df_resultados['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de Random Forest
modelo = RandomForestRegressor()
modelo.fit(X_train, y_train)

RandomForestRegressor()

In [7]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2023, 3),
                            'mes': range(10, 13)})
fechas_20232024 = pd.DataFrame({'año': np.repeat(2024, 1),
                            'mes': range(1, 2)})
fechas_2024 = pd.DataFrame({'año': np.repeat(2024, 4),
                            'mes': range(9, 13)})
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2023,fechas_20232024,fechas_2024, fechas_2025_2026])  

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones
total_pachuca = pd.concat([df_resultados, nuevas_fechas], ignore_index=True)
total_todos_pachuca = total_pachuca.groupby(['año', 'mes'])['montos'].median().reset_index()

# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
# Formatear la columna 'montos' con la función personalizada
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].map(format_number)
#total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float).astype(int)
# Convertir la columna 'año' a tipo int
total_todos_pachuca['año'] = total_todos_pachuca['año'].astype(int)
# Convertir la columna 'montos' a tipo float
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float)
total_todos_pachuca

,año,mes,montos
0,2022,10,999999.99
1,2022,11,999999.99
2,2023,9,999999.99
3,2023,10,1177999.99
4,2023,11,1177999.99
5,2023,12,1177999.99
6,2024,1,2421000.00
7,2024,2,2500000.00
8,2024,3,2500000.00
9,2024,4,2300000.00


In [8]:
meses_dict_invertido = {v: k for k, v in meses_dict.items()}
total_todos_pachuca['label'] = total_todos_pachuca['mes'].apply(lambda x: meses_dict_invertido[x]) + '-' + total_todos_pachuca['año'].astype(str)
total_todos_pachuca = total_todos_pachuca.sort_values(by=['año', 'mes'])
total_todos_pachuca

,año,mes,montos,label
0,2022,10,999999.99,octubre-2022
1,2022,11,999999.99,noviembre-2022
2,2023,9,999999.99,septiembre-2023
3,2023,10,1177999.99,octubre-2023
4,2023,11,1177999.99,noviembre-2023
5,2023,12,1177999.99,diciembre-2023
6,2024,1,2421000.00,enero-2024
7,2024,2,2500000.00,febrero-2024
8,2024,3,2500000.00,marzo-2024
9,2024,4,2300000.00,abril-2024


In [9]:
# Datos
año = total_todos_pachuca['label']
promedio = total_todos_pachuca['montos']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=año,
    y=promedio,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for a, p in zip(año, promedio):
    fig.add_annotation(
        x=a,
        y=p,
        text=f"${p:,.2f}",  # Formatear el valor del promedio como moneda
        showarrow=False,
        font=dict(color='black', size=12),
        xshift=0,
        yshift=17,
        textangle=0
    )
# Actualizar diseño
fig.update_layout(
    #title='Precio Media Anual',
    yaxis=dict(
        range=[900000, 5500000],  # Establecer el rango del eje y un poco antes del millón
        tickvals=[1900000, 2900000, 3900000, 5000000],  # Definir los valores de las marcas en el eje y un poco antes del millón
        ticktext=['$1,900,000.00','$2,900,000.00', '$3,900,000.00','$5,500,000.00'],  # Definir el texto de las marcas en el eje y un poco antes del millón
        gridcolor='#dddcda',   # Color de las líneas de la cuadrícula
        gridwidth=1  # Ancho de las líneas de la cuadrícula
    ),margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    xaxis=dict(
        gridcolor='#dddcda', 
        tickmode='array',
        tickvals=año,
        ticktext=año
    ),
    plot_bgcolor='rgba(0,0,0,0)'
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Exportar la gráfica a un archivo HTML
    fig_html = fig.to_html()
    with open(os.path.join(carpeta, nombre_archivo + '.html'), 'w') as archivo:
        archivo.write(fig_html)
guardar_grafico_como_html(fig, 'g_scatt_precio_medianual', carpeta='assets/graficas')
fig.show()

# Regresión mensual (RF)
## Cortando 2023

In [11]:
df_resultados = df_resultados[~df_resultados['año'].isin([2022, 2023])]
df_resultados = df_resultados.groupby(['mes','año'])['precio'].median().reset_index()
df_resultados.rename(columns={'precio': 'montos'}, inplace=True)

df_resultados

,mes,año,montos
0,2,2024,2500000.0
1,3,2024,2500000.0
2,4,2024,2300000.0
3,5,2024,2250000.0
4,6,2024,2300000.0
5,7,2024,2400000.0
6,8,2024,2500000.0


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np


# Preparar datos para el modelo
X = df_resultados[['año', 'mes']]
y = df_resultados['montos']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de Random Forest
modelo = RandomForestRegressor()
modelo.fit(X_train, y_train)

RandomForestRegressor()

In [13]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_20232024 = pd.DataFrame({'año': np.repeat(2024, 1),
                            'mes': range(1, 2)})
fechas_2024 = pd.DataFrame({'año': np.repeat(2024, 4),
                            'mes': range(9, 13)})
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_20232024,fechas_2024, fechas_2025_2026])  

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones
total_pachuca = pd.concat([df_resultados, nuevas_fechas], ignore_index=True)
total_todos_pachuca = total_pachuca.groupby(['año', 'mes'])['montos'].median().reset_index()

# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
# Formatear la columna 'montos' con la función personalizada
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].map(format_number)
#total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float).astype(int)
# Convertir la columna 'año' a tipo int
total_todos_pachuca['año'] = total_todos_pachuca['año'].astype(int)
# Convertir la columna 'montos' a tipo float
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float)
total_todos_pachuca

,año,mes,montos
0,2024,1,2286500.0
1,2024,2,2500000.0
2,2024,3,2500000.0
3,2024,4,2300000.0
4,2024,5,2250000.0
5,2024,6,2300000.0
6,2024,7,2400000.0
7,2024,8,2500000.0
8,2024,9,2471000.0
9,2024,10,2471000.0


In [14]:
meses_dict_invertido = {v: k for k, v in meses_dict.items()}
total_todos_pachuca['label'] = total_todos_pachuca['mes'].apply(lambda x: meses_dict_invertido[x]) + '-' + total_todos_pachuca['año'].astype(str)
total_todos_pachuca = total_todos_pachuca.sort_values(by=['año', 'mes'])
total_todos_pachuca

,año,mes,montos,label
0,2024,1,2286500.0,enero-2024
1,2024,2,2500000.0,febrero-2024
2,2024,3,2500000.0,marzo-2024
3,2024,4,2300000.0,abril-2024
4,2024,5,2250000.0,mayo-2024
5,2024,6,2300000.0,junio-2024
6,2024,7,2400000.0,julio-2024
7,2024,8,2500000.0,agosto-2024
8,2024,9,2471000.0,septiembre-2024
9,2024,10,2471000.0,octubre-2024


In [15]:
# Datos
año = total_todos_pachuca['label']
promedio = total_todos_pachuca['montos']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=año,
    y=promedio,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for a, p in zip(año, promedio):
    fig.add_annotation(
        x=a,
        y=p,
        text=f"${p:,.2f}",  # Formatear el valor del promedio como moneda
        showarrow=False,
        font=dict(color='black', size=12),
        xshift=0,
        yshift=17,
        textangle=0
    )
# Actualizar diseño
fig.update_layout(
    #title='Precio Media Anual',
    yaxis=dict(
        range=[2000000, 3000000],  # Establecer el rango del eje y un poco antes del millón
        tickvals=[2000000, 2500000, 3000000],  # Añadir la mitad entre 2000000 y 3000000
        ticktext=['$2,000,000.00','$2,500,000.00','$3,000,000.00'],  # Añadir el texto para la marca de la mitad
        gridcolor='#dddcda',   # Color de las líneas de la cuadrícula
        gridwidth=1  # Ancho de las líneas de la cuadrícula
    ),margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    xaxis=dict(
        gridcolor='#dddcda', 
        tickmode='array',
        tickvals=año,
        ticktext=año
    ),
    plot_bgcolor='rgba(0,0,0,0)'
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Exportar la gráfica a un archivo HTML
    fig_html = fig.to_html()
    with open(os.path.join(carpeta, nombre_archivo + '.html'), 'w') as archivo:
        archivo.write(fig_html)
guardar_grafico_como_html(fig, 'g_scatt_precio_medianual', carpeta='assets/graficas')
fig.show()

In [ ]:
total_todos_pachuca

In [34]:
todos_year = total_todos_pachuca[~total_todos_pachuca['año'].isin([2022, 2023])].groupby('año')['montos'].mean().reset_index()
todos_year

,año,montos
0,2024,3007466.07
1,2025,4203316.92
2,2026,5319397.61


In [35]:
# Datos
año = todos_year['año']
promedio = todos_year['montos']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=año,
    y=promedio,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for a, p in zip(año, promedio):
    fig.add_annotation(
        x=a,
        y=p,
        text=f"${p:,.2f}",  # Formatear el valor del promedio como moneda
        showarrow=False,
        font=dict(color='black', size=12),
        xshift=0,
        yshift=17,
        textangle=0
    )
# Actualizar diseño
fig.update_layout(
    #title='Precio Media Anual',
    yaxis=dict(
       
        gridcolor='#dddcda',   # Color de las líneas de la cuadrícula
        gridwidth=1  # Ancho de las líneas de la cuadrícula
    ),margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    xaxis=dict(
        gridcolor='#dddcda', 
        tickmode='array',
        tickvals=año,
        ticktext=año
    ),
    plot_bgcolor='rgba(0,0,0,0)'
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Exportar la gráfica a un archivo HTML
    fig_html = fig.to_html()
    with open(os.path.join(carpeta, nombre_archivo + '.html'), 'w') as archivo:
        archivo.write(fig_html)
guardar_grafico_como_html(fig, 'g_scatt_precio_medianual_anual', carpeta='assets/graficas')
fig.show()

# Regresión Anual

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np


# Preparar datos para el modelo
X = df_resultados[['año', 'mes']]
y = df_resultados['precio']
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [23]:
df_resultados = df_resultados.groupby('año')['precio'].median().reset_index()
df_resultados.rename(columns={'precio': 'montos'}, inplace=True)

df_resultados

,año,montos
0,2022,999999.99
1,2023,999999.99
2,2024,2400000.0


In [24]:
# DF con todas las combinaciones posibles de año y mes para 2023 FALTANTES (AUG-DEC)
fechas_2023 = pd.DataFrame({'año': np.repeat(2024, 6),
                            'mes': range(7, 13)})
fechas_2025_2026 = pd.DataFrame({'año': np.repeat(range(2025, 2027), 12),
                                  'mes': np.tile(range(1, 13), 2)})
nuevas_fechas = pd.concat([fechas_2023, fechas_2025_2026])  

# Realizamos predicciones con el modelo entrenado
predicciones = modelo.predict(nuevas_fechas)
# Agregamos las predicciones al DataFrame de nuevas fechas
nuevas_fechas['montos'] = predicciones
total_pachuca = pd.concat([df_resultados, nuevas_fechas], ignore_index=True)
total_todos_pachuca = total_pachuca.groupby('año')['montos'].median().reset_index()

# Función para formatear números con separadores de coma y dos decimales
def format_number(x):
   return '{:,.2f}'.format(x)
# Formatear la columna 'montos' con la función personalizada
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].map(format_number)
#total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float).astype(int)
# Convertir la columna 'año' a tipo int
total_todos_pachuca['año'] = total_todos_pachuca['año'].astype(int)
# Convertir la columna 'montos' a tipo float
total_todos_pachuca['montos'] = total_todos_pachuca['montos'].replace({',': ''}, regex=True).astype(float)
total_todos_pachuca

,año,montos
0,2022,999999.99
1,2023,999999.99
2,2024,3007466.07
3,2025,4203316.92
4,2026,5319397.61


In [25]:
# Datos
año = total_todos_pachuca['año']
promedio = total_todos_pachuca['montos']

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=año,
    y=promedio,
    mode='markers+lines',
    marker=dict(color='blue'),
))
# Agregar anotaciones para mostrar los valores encima de los puntos
for a, p in zip(año, promedio):
    fig.add_annotation(
        x=a,
        y=p,
        text=f"${p:,.2f}",  # Formatear el valor del promedio como moneda
        showarrow=False,
        font=dict(color='black', size=12),
        xshift=0,
        yshift=17,
        textangle=0
    )
# Actualizar diseño
fig.update_layout(
    #title='Precio Media Anual',
    yaxis=dict(
        range=[900000, 5500000],  # Establecer el rango del eje y un poco antes del millón
        tickvals=[1900000, 2900000, 3900000, 5000000],  # Definir los valores de las marcas en el eje y un poco antes del millón
        ticktext=['$1,900,000.00','$2,900,000.00', '$3,900,000.00','$5,500,000.00'],  # Definir el texto de las marcas en el eje y un poco antes del millón
        gridcolor='#dddcda',   # Color de las líneas de la cuadrícula
        gridwidth=1  # Ancho de las líneas de la cuadrícula
    ),margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
    xaxis=dict(
        gridcolor='#dddcda', 
        tickmode='array',
        tickvals=año,
        ticktext=año
    ),
    plot_bgcolor='rgba(0,0,0,0)'
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Exportar la gráfica a un archivo HTML
    fig_html = fig.to_html()
    with open(os.path.join(carpeta, nombre_archivo + '.html'), 'w') as archivo:
        archivo.write(fig_html)
guardar_grafico_como_html(fig, 'g_scatt_precio_medianual', carpeta='assets/graficas')
fig.show()